# graphAttack Tutorial #01

## Introduction

This tutorial demonstrates the basic graph framework used in graphAttack and applies it to a simple linear model

You should be familiar with basic linear algebra, Python and the Jupyter Notebook editor. It also helps if you have a basic understanding of Machine Learning and classification.

## Imports

In [51]:
import numpy as np
import graphAttack as ga
import warnings
warnings.filterwarnings('ignore')

This was developed using Python 3.6.1

# Nodes

graphAttack uses Nodes ot store data instead of normal python variables, the Nodes are classified into

### Variables

Those objects are used to store numerical data to be fed into the graph. They use numpy arrays to store data. Here we initialize some variables: 

In [11]:
a = np.array([2])
b = np.array([3])

aVar = ga.Variable(a)
bVar = ga.Variable(b)

We can now extract the stored data by calling:

In [12]:
aData = aVar.getValue()
print(aVar.getValue())

[2]


and print the information about a variable by:

In [13]:
print(aVar)

<Variable with outputs: ()>


### Operations

Operatins usually take as input other Nodes (Variables or Operations) and can be used to extract the output of underlying mathematical operation. Lets start with simple addition:

In [26]:
addOp = ga.AddOperation(aVar, bVar)

Again, we can obtain the value of the operation by .getValue() as well as some information about it by printing it:

In [27]:
print("2 + 3 = ", addOp.getValue())
print("Details of addOp:", addOp)

2 + 3 =  [5]
Details of addOp: <AddOperation with inputs: (Variable, Variable) and outputs: ()>


Defining operations could also be done using operators:

In [28]:
addOp2 = aVar + bVar
print("2 + 3 = ", addOp2.getValue())
print("Details of addOp2:", addOp2)

2 + 3 =  [5]
Details of addOp2: <AddOperation with inputs: (Variable, Variable) and outputs: ()>


We could chain operations together to get something slightly more complicated

In [29]:
aVar = ga.Variable(np.array([2]))
bVar = ga.Variable(np.array([3]))
cVar = ga.Variable(np.array([4]))

addOp = ga.AddOperation(aVar, bVar)
multOp = ga.MultiplyOperation(addOp, cVar)

print("(2 + 3) * 4 = ", multOp.getValue())

(2 + 3) * 4 =  [20]


Again, same could be achieved wiht operators:

In [30]:
aVar = ga.Variable(np.array([2]))
bVar = ga.Variable(np.array([3]))
cVar = ga.Variable(np.array([4]))

addOp = aVar + bVar
multOp = addOp * cVar

print("(2 + 3) * 4 = ", multOp.getValue())

(2 + 3) * 4 =  [20]


# Differentiation

graphAttack supports automatic differentiation to ease up the burden of optimising your parameters, lets look at our previous example, however this time, we could ask for a derevative of one of the Variables with respect to the final operation of the graph:

In [38]:
aVar = ga.Variable(np.array([2]))
bVar = ga.Variable(np.array([3]))
cVar = ga.Variable(np.array([4]))

addOp = aVar + bVar
multOp = addOp * cVar

print("(2 + 3) * 4 = ", multOp.getValue())
print("The derevatve with respect to cVar is:", cVar.getGradient())

(2 + 3) * 4 =  [20]
The derevatve with respect to cVar is: [ 5.]


Each time we ask for a gradient, it looks for the derevatives up the operations chain untill it reaches a dead end and then uses a chain rule to propagate the gradients backwards through the whole graph. Normally we would ask for derevatives of variables but we also could find the derevative of an individual operations as long as we provide the input with repsect to which we want the derivation to be made.

In [39]:
print("The derevatve of multOp respect to cVar is:", multOp.getGradient(cVar))

The derevatve of multOp respect to cVar is: [ 5.]


# Graph

Quite an important part of this library is the graph object, it helps store and organise your operations, it can be initialized by

In [40]:
mainGraph = ga.Graph()

Now we can add more operations to it using the .addOperation method, it has a couple of parameters:

In [45]:
def addOperationDUMMY(self,
                     operation,
                     doGradient=False,
                     finalOperation=False,
                     feederOperation=False):
    """Add an operation ot the graph
    
    Parameters
    ----------
    operation : ga.Operation
        Operation or Variable to be added
    doGradient : bool
        When true, calculate the derevative of the final operation with respect to this
        operation when self.getGradients() is called
    finalOperation : bool
        When true, specify this operation as final of the graph
    feederOperation : bool
        When true, specify this operation as the data feeder operation
        
    Returns
    -------
    ga.Operation
        the handle for added operation
        
    Raises
    ------
    ValueError
        Graph can only provide gradients with respect to variables!
        Only variables can be feeders
    """

In [69]:
mainGraph = ga.Graph()

a = np.array([2])
b = np.array([3])
c = np.array([4])

aVar = mainGraph.addOperation(ga.Variable(a), doGradient=True)
bVar = mainGraph.addOperation(ga.Variable(b), doGradient=True)
addOp = mainGraph.addOperation(aVar + bVar)
cVar = mainGraph.addOperation(ga.Variable(c), doGradient=True)
multOp = mainGraph.addOperation(addOp * cVar, finalOperation=True)

print(mainGraph)

Computation Graph:
<op0-Variable with outputs: (op2-AddOperation, )>
<op1-Variable with outputs: (op2-AddOperation, )>
<op2-AddOperation with inputs: (op0-Variable, op1-Variable) and outputs: (op4-MultiplyOperation, )>
<op3-Variable with outputs: (op4-MultiplyOperation, )>
<op4-MultiplyOperation with inputs: (op2-AddOperation, op3-Variable) and outputs: ()>



We cana now ask for Value of the final operation and gradients with respect to the specified variables:

In [73]:
print("Final value of (2 + 3) * 4 = ", mainGraph.getValue())
print("Gradients are:")
for g in mainGraph.getGradients():
    print(g)

Final value of (2 + 3) * 4 =  [20]
Gradients are:
(0, 'op0-Variable', array([ 4.]))
(1, 'op1-Variable', array([ 4.]))
(3, 'op3-Variable', array([ 5.]))


# Linear Regression

Lets try to use the library to run a simple linear regression model that will extract the function:
y = 2 * x1 + 3 * x2 + 4

Initially we need to prepare the dataset in the rights shapes, the shapes are important and it is worth to keep track of them since often in python column vectors end up being row vectors

In [183]:
x = np.random.random((100, 2))
y = (x[:, 0] * 2 + x[:, 1] * 3 + 4).reshape(-1, 1)

Next, we need to compose a graph that will represent our model

In [184]:
mainGraph = ga.Graph()

w = np.random.random((1, 2)).T
b = np.random.random(1)

print(w, b)
feed = mainGraph.addOperation(ga.Variable(x), feederOperation=True)
wop = mainGraph.addOperation(ga.Variable(w), doGradient=True)
matmulop = mainGraph.addOperation(feed @ wop)
bop = mainGraph.addOperation(ga.Variable(b), doGradient=True)
addop = mainGraph.addOperation(matmulop + bop)
costop = mainGraph.addOperation(ga.QuadratiCcostOperation(addop, y), finalOperation=True)

print(mainGraph)

[[ 0.16786996]
 [ 0.11129035]] [ 0.02665591]
Computation Graph:
<op0-Variable with outputs: (op2-MatMatmulOperation, )>
<op1-Variable with outputs: (op2-MatMatmulOperation, )>
<op2-MatMatmulOperation with inputs: (op0-Variable, op1-Variable) and outputs: (op4-AddOperation, )>
<op3-Variable with outputs: (op4-AddOperation, )>
<op4-AddOperation with inputs: (op2-MatMatmulOperation, op3-Variable) and outputs: (op5-QuadratiCcostOperation, )>
<op5-QuadratiCcostOperation with input: (op4-AddOperation) and outputs: ()>



We can find the cost and gradients with respect to our parameters by:

In [185]:
print("Cost:", mainGraph.getValue())
print(costop.nExamples)
print("Gradients are:")
for g in mainGraph.getGradients():
    print(g)

Cost: 20.2874106085
100
Gradients are:
(1, 'op1-Variable', array([[-2.97423341],
       [-3.49364679]]))
(3, 'op3-Variable', array([-6.30036239]))


We can figure out that the random parameters are not terrific, in order to optimize them we need ot construct a function that would return cost and flattened gradients so that we can feed it into a minimizer of choice, the code below is explained soon

In [186]:
def fprime(p, data, labels):
    mainGraph.feederOperation.assignData(data)
    mainGraph.costOperation.assignLabels(labels)
    mainGraph.attachParameters(p)
    mainGraph.resetAll()
    c = mainGraph.getValue()
    mainGraph.getGradients()
    g = mainGraph.unrollGradients()
    return c, g

Function fprime takes as input a flattened vector of parameters, the data and labels. While data and labels are quite self-explanatory, lets discuss the parameters vector. It is a flattened vector containing all the weigths and biases. It can be extracted from a graph by calling mainGraph.unrollGraientParameters() and attached to a graph (values of weigths/biases variables updated) by mainGraph.attachParameters(p).



In [187]:
print("Current parameters", wop.getValue(), bop.getValue())
param0 = mainGraph.unrollGradientParameters()
print("Unrolled parameters", param0)

Current parameters [[ 0.16786996]
 [ 0.11129035]] [ 0.02665591]
Unrolled parameters [ 0.16786996  0.11129035  0.02665591]


Inside the function, we assign the data to the feeder Variable, assign labels to the cost Variable, attach provided parameters and reset the values of all operations (not Variables) so that the flow can be re-computed. Later the final cost and gredients is extracted from the graph with the gradients being unrolled ot match the parameters vector

In [188]:
print("Gradients are:")
for g in mainGraph.getGradients():
    print(g)
print("Unrolled gradients", mainGraph.unrollGradients())

Gradients are:
(1, 'op1-Variable', array([[-2.97423341],
       [-3.49364679]]))
(3, 'op3-Variable', array([-6.30036239]))
Unrolled gradients [-2.97423341 -3.49364679 -6.30036239]


We can pass our function to a minimizer of choice, for now lets use the one in scipy

In [190]:
import scipy.optimize
res = scipy.optimize.minimize(fprime,
                              param0,
                              args=(x, y),
                              method="BFGS",
                              jac=True)
print(res)

      fun: 9.847984166654734e-12
 hess_inv: array([[ 12.10980509,   0.39023944,  -5.64093334],
       [  0.39023944,  12.28600029,  -6.53002646],
       [ -5.64093334,  -6.53002646,   6.9053932 ]])
      jac: array([ -1.75466966e-07,   7.62098485e-07,  -8.12225231e-07])
  message: 'Optimization terminated successfully.'
     nfev: 12
      nit: 9
     njev: 12
   status: 0
  success: True
        x: array([ 2.00000294,  3.0000156 ,  3.99998975])


# SGD

In ML models, it is often useful to use stocastic gradient descent as your minimizer due to large number of datapoints. GraphAttack comes with a simple implementation of the Adam method that is considered quite robust. The details of the implementation can be found in
https://arxiv.org/pdf/1412.6980.pdf

Here we will simply use the minimizer, it can be initialized by

In [216]:
adamGrad = ga.adaptiveSGD(trainingData=x,
                          trainingLabels=y,
                          param0=param0,
                          epochs=1e3,
                          miniBatchSize=5,
                          initialLearningRate=1e-3,
                          beta1=0.9,
                          beta2=0.999,
                          epsilon=1e-8,
                          testFrequency=1e1,
                          function=fprime)

With the explanation below:

The minimalization is performed by calling

In [217]:
params = adamGrad.minimize(printTrainigCost=True)
print(params)

Mibatch: 0 out of 20 from epoch: 0 out of 1000, iterCost is: 1.260852e-02
Mibatch: 0 out of 20 from epoch: 100 out of 1000, iterCost is: 1.101159e+01
Mibatch: 0 out of 20 from epoch: 200 out of 1000, iterCost is: 1.813191e+00
Mibatch: 0 out of 20 from epoch: 300 out of 1000, iterCost is: 1.150450e-01
Mibatch: 0 out of 20 from epoch: 400 out of 1000, iterCost is: 3.079652e-02
Mibatch: 0 out of 20 from epoch: 500 out of 1000, iterCost is: 9.495015e-03
Mibatch: 0 out of 20 from epoch: 600 out of 1000, iterCost is: 1.028337e-03
Mibatch: 0 out of 20 from epoch: 700 out of 1000, iterCost is: 9.837792e-06
Mibatch: 0 out of 20 from epoch: 800 out of 1000, iterCost is: 1.150255e-10
Mibatch: 0 out of 20 from epoch: 900 out of 1000, iterCost is: 3.864496e-24
[ 2.  3.  4.]


## License (MIT)

Copyright (c) 2016 by [Magnus Erik Hvass Pedersen](http://www.hvass-labs.org/)

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.